In [ ]:
data_dir = ".\\data\\"

# SLC-2445, vac, 72hrs
filename = "2018-07-27-14-45-18.tdms"
keyence_filename= None

# SLC-1730, vac, 24hrs
filename = "2018-07-26-14-39-24.tdms"
keyence_filename= None

# SLC-1730, vac, 48hrs
filename = "2018-07-30-16-19-44.tdms"
keyence_filename= None

In [ ]:
import batch_utils
import data_tools

In [ ]:
from ipywidgets import FloatProgress
from IPython.display import display

import numpy as np
from pprint import pprint
import datetime
from time import time

from importlib import reload
reload(batch_utils)
reload(data_tools)

full_data, info = batch_utils.load_data(filename, keyence_filename, suffix="full_data") 
pprint(info)
batch_utils.print_lengths(full_data, "Full dataset:")

In [ ]:

if filename == "2018-07-26-14-39-24.tdms":  
    breaks = ('set_point', 'temperature', 'position')
    for key in breaks:
        print("\nLooking for the break in", key)
        time_diff = [(b-a) for a, b in zip(full_data[key]['time'], full_data[key]['time'][1:])]

        print([
            ("Max ", np.max(time_diff).total_seconds(), time_diff.index(np.max(time_diff))), 
            ("Mean", np.mean(time_diff).total_seconds()),
            ("Min ", np.min(time_diff).total_seconds(), time_diff.index(np.min(time_diff)))
        ])
        
        broken = time_diff.index(np.min(time_diff))

        print(key, "broken at", broken, time_diff[broken].total_seconds())
        
        diff = time_diff[broken] - time_diff[broken +1]
        
        fixed = []
        fixed.extend(full_data[key]['time'][:broken+1])
        fixed.extend([(v - diff) for v in full_data[key]['time'][broken+1:]])
                        
        print("Fixed", key)
        
        time_diff = [(b-a) for a, b in zip(fixed, fixed[1:])]

        print([
            ("Max ", np.max(time_diff).total_seconds(), time_diff.index(np.max(time_diff))), 
            ("Mean", np.mean(time_diff).total_seconds()),
            ("Min ", np.min(time_diff).total_seconds(), time_diff.index(np.min(time_diff)))
        ])
        
        full_data[key]['time'] = fixed

In [ ]:
limits = [full_data['set_point']['time'][48000], full_data['set_point']['time'][50000]]

data = dict(zip(full_data.keys(), [data_tools.truncate(limits, full_data[k], find_mode={'seek':False}) for k in full_data]))
batch_utils.print_lengths(data, "Truncated dataset:")

data=full_data

batch_utils.print_structure(data)

In [ ]:
# Initialise the index
indices = dict.fromkeys(data, 0)

# Start when all data streams have begun
current_time = max([data[d]['time'][0] for d in data])
print(current_time,)

def update_indices(new_time):
    change = set()
    for index in indices:
        try:
            while data[index]['time'][indices[index]+1] <= new_time:
                indices[index] += 1
                change.add(index)
        except IndexError:
            break
    return change

update_indices(current_time)

print(indices)        
pprint([str(data[index]['time'][indices[index]]) for index in indices])

# Sanity check all next times are greater than the current time
for index in indices:
    print(data[index]['time'][indices[index]+1] > current_time)

end = sum([len(data[d]['time']) for d in data])

print(end, "points to evaluate")

progress = FloatProgress(value=0, max=end/1000)
display(progress)

In [ ]:
# Initialise the stats...

data['set_point']['trans'] =        [None,] * len(data['set_point']['time'])
data['set_point']['avg_position'] = [None,] * len(data['set_point']['time'])
data['set_point']['min_position'] = [None,] * len(data['set_point']['time'])
data['set_point']['max_position'] = [None,] * len(data['set_point']['time'])

#data['position']['round'] =         [None,] * len(data['position']['time'])
data['position']['set_point'] =     [None,] * len(data['position']['time'])
data['position']['mask'] =          [None,] * len(data['position']['time'])

mask_next = datetime.timedelta(seconds = 0.5)
mask_prev = datetime.timedelta(seconds = 0.075)

pos_index_last_sp = indices['position']

pos_list = list()

while sum([indices[index] for index in indices]) < end:
    
    # Position - setpoint
    data['position']['set_point'][indices['position']] = \
        data['position']['data'][indices['position']] \
        - data['set_point']['data'][indices['set_point']]
        
        
    # Aggregate reset
    if current_time == data['set_point']['time'][indices['set_point']]:
        # New set point value - save the values and reset!
        try:
            data['set_point']['avg_position'][indices['set_point']-1] = np.mean(pos_list)
            data['set_point']['min_position'][indices['set_point']-1] = np.min(pos_list)
            data['set_point']['max_position'][indices['set_point']-1] = np.max(pos_list)
        except ValueError:
            print("Can't aggregate!")
            pprint(indices)
        
        pos_list = list()
    
    
    # Mask
    try:
        data['position']['mask'][indices['position']] = (
            (data['position']['time'][indices['position']]) \
            < (data['set_point']['time'][indices['set_point']+1] - mask_next)) \
        and (
            (data['position']['time'][indices['position']]) \
            > (data['set_point']['time'][indices['set_point']] - mask_prev))
        
        pass
    except IndexError as e:
        # Run out of set_points
        next_set_time =    data['set_point']['time'][indices['set_point']] \
                        + (data['set_point']['time'][indices['set_point']] \
                         - data['set_point']['time'][indices['set_point']-1])
        
        data['position']['mask'][indices['position']] = (
            (data['position']['time'][indices['position']]) \
            < (next_set_time - mask_next)) \
        and (
            (data['position']['time'][indices['position']]) \
            > (data['set_point']['time'][indices['set_point']] - mask_prev))
    
    
    
    
    next_time = min([data[index]['time'][min([indices[index]+1, len(data[index]['time'])-1])] for index in indices])
    
    old_indices = indices.copy()
    change = update_indices(next_time)
    
    # Aggregate updating
    if 'position' in change and not data['position']['mask'][old_indices['position']]:
        pos_list.append(data['position']['set_point'][old_indices['position']])
    
    
    #for index in indices:
    #    print(index, old_indices[index], indices[index], old_indices[index] == indices[index])
        
    if all([old_indices[index] == indices[index] for index in indices]):
        print("Stopping at {}".format(indices))
        break
    
    #print(sum([indices[index] for index in indices]), end)
    #print(current_time)
    #print(next_time)
    #print()
    
    current_time = next_time
    
    if sum([indices[index] for index in indices]) % 1000 == 0:
        progress.value += 1
    
    #break
    
progress.value = progress.max
print("Done!")

In [ ]:
print("Storing the processed data")
batch_utils.store_data(data, filename, keyence_filename, suffix="seq", info=info)
print("Done!")

data.pop('info')